Finding best architectures using Keras Tuner

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 5.5 MB/s 
     |████████████████████████████████| 1.6 MB 40.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
import keras_tuner as kt
df = pd.read_csv("drive/MyDrive/Engineer's Project/output_pln_usd.csv")

In [ ]:
df.pop('usa_cpi')
df.pop('pol_cpi')
df.pop('usa_inter')
df.pop('pol_inter')
df.pop('Date')

0       2010.11.15
1       2010.11.16
2       2010.11.17
3       2010.11.18
4       2010.11.19
           ...    
3537    2022.03.27
3538    2022.03.28
3539    2022.03.29
3540    2022.03.30
3541    2022.03.31
Name: Date, Length: 3542, dtype: object

In [ ]:
scaler = MinMaxScaler()
df_numpy = df.to_numpy() 
#len(df_numpy[:2500])
scaler1 = scaler.fit(df_numpy[:3000])
df_scalled = scaler1.transform(df_numpy)
#df_scaled_all = scaler.
df_scalled = pd.DataFrame(df_scalled, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
lookback = 15
step = 1
delay = 0
batch_size = 128

In [ ]:
float_data = np.array(df_scalled).astype('float32')

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] 
    yield samples, targets

In [ ]:
train_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=2500,
#shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
val_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=2501,
max_index=3000,
step=step,
batch_size=batch_size)

In [ ]:
test_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=3001,
max_index=3541,
step=step,
batch_size=batch_size)

In [ ]:
train_steps = (2501 - lookback)
val_steps = (3001 - 2501 - lookback)
test_steps = (3543 - 3001 - lookback)

In [ ]:
def model_builder(hp):
  #hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3])
  hp_units = hp.Int('units GRU', min_value=130, max_value = 150, step=5)
  hp_units2 = hp.Int('units Dense', min_value=110, max_value=130,step=5)
  model = tf.keras.models.Sequential()
  model.add(layers.GRU(units=hp_units,
    #activation=hp.Choice('lstm_activation2',values=['relu', 'sigmoid','softmax']),
    activation='sigmoid',
   # recurrent_dropout=0.1,
   # dropout=0.15,
    input_shape=(None, float_data.shape[-1])))
  model.add(layers.Dropout(hp.Float('Dropout rate',min_value=0,max_value=0.3,step=0.05)))
  model.add(layers.Dense(units=hp_units2,
    #activation=hp.Choice('dense_activation',values=['relu', 'sigmoid','softmax'])))
    activation='sigmoid'))
  model.add(layers.Dense(1,
    ##activation = hp.Choice('last_activ',values=['relu', 'sigmoid','softmax'])))
    activation='sigmoid'))
  model.compile(optimizer= tf.keras.optimizers.RMSprop(learning_rate=0.01), loss='mae')
  return model

In [ ]:
tuner = kt.RandomSearch(model_builder,
    objective = 'val_loss',
    directory='my_dir',
    project_name='intro_to_kt_All',
    max_trials = 20
)

In [ ]:
tunerB = kt.BayesianOptimization(model_builder,
    objective='val_loss',
    max_trials=20)
#GRU

In [ ]:
tunerB2 = kt.BayesianOptimization(model_builder,
    objective='val_loss',
    max_trials=60,
    directory='lstm')
#LSTM

In [ ]:
tunerB3 = kt.BayesianOptimization(model_builder,
    objective='val_loss',
    max_trials=36,
    directory='lstm')
#LSTM

In [ ]:
tunerHyper = kt.Hyperband(model_builder,
    objective='val_loss',
    directory='hyper2')

In [ ]:
tunerHyper.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=20,
    steps_per_epoch = 200,
)

Trial 87 Complete [00h 00m 14s]
val_loss: 0.026919124647974968

Best val_loss So Far: 0.015270381234586239
Total elapsed time: 00h 03m 15s

Search: Running Trial #88

Value             |Best Value So Far |Hyperparameter
0.001             |0.01              |learning_rate
120               |80                |units GRU
140               |120               |units Dense
0.4               |0                 |Dropout rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
 33/200 [===>..........................] - ETA: 5s - loss: 0.3800

KeyboardInterrupt: ignored

In [ ]:
tuner.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 13 Complete [00h 16m 25s]
val_loss: 0.20283201336860657

Best val_loss So Far: 0.02437911368906498
Total elapsed time: 02h 26m 24s
INFO:tensorflow:Oracle triggered exit


LSTM neurons

In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt
Showing 10 best trials
Trial summary
Hyperparameters:
units: 128
Score: 0.02437911368906498
Trial summary
Hyperparameters:
units: 160
Score: 0.027353744953870773
Trial summary
Hyperparameters:
units: 96
Score: 0.030319377779960632
Trial summary
Hyperparameters:
units: 64
Score: 0.032807569950819016
Trial summary
Hyperparameters:
units: 224
Score: 0.03391774371266365
Trial summary
Hyperparameters:
units: 192
Score: 0.03637487813830376
Trial summary
Hyperparameters:
units: 256
Score: 0.05576149374246597
Trial summary
Hyperparameters:
units: 448
Score: 0.058860041201114655
Trial summary
Hyperparameters:
units: 416
Score: 0.07183432579040527
Trial summary
Hyperparameters:
units: 384
Score: 0.08969735354185104


LSTM activation

In [ ]:
tuner.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 3 Complete [00h 03m 29s]
val_loss: 0.03626972809433937

Best val_loss So Far: 0.017951438203454018
Total elapsed time: 00h 10m 20s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt_llstmActiv
Showing 10 best trials
Trial summary
Hyperparameters:
lstm_activation: sigmoid
Score: 0.017951438203454018
Trial summary
Hyperparameters:
lstm_activation: relu
Score: 0.022853845730423927
Trial summary
Hyperparameters:
lstm_activation: softmax
Score: 0.03626972809433937


Dense Activattion

In [ ]:
tuner.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 2 Complete [00h 03m 25s]
val_loss: 0.019377298653125763

Best val_loss So Far: 0.019377298653125763
Total elapsed time: 00h 07m 52s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt_DenseActiv
Showing 10 best trials
Trial summary
Hyperparameters:
dense_activation: sigmoid
Score: 0.019377298653125763
Trial summary
Hyperparameters:
dense_activation: softmax
Score: 0.020196324214339256


Dense neurons

In [ ]:
tuner.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 14 Complete [00h 03m 25s]
val_loss: 0.012959214858710766

Best val_loss So Far: 0.012670903466641903
Total elapsed time: 00h 47m 36s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt_DenseNeurons
Showing 10 best trials
Trial summary
Hyperparameters:
units: 14
Score: 0.012670903466641903
Trial summary
Hyperparameters:
units: 26
Score: 0.012699891813099384
Trial summary
Hyperparameters:
units: 6
Score: 0.012891581282019615
Trial summary
Hyperparameters:
units: 28
Score: 0.012901070527732372
Trial summary
Hyperparameters:
units: 18
Score: 0.012959214858710766
Trial summary
Hyperparameters:
units: 12
Score: 0.013039239682257175
Trial summary
Hyperparameters:
units: 24
Score: 0.013063455000519753
Trial summary
Hyperparameters:
units: 4
Score: 0.013514241203665733
Trial summary
Hyperparameters:
units: 10
Score: 0.014631305821239948
Trial summary
Hyperparameters:
units: 8
Score: 0.015606595203280449


Dropout values

In [ ]:
tuner.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 11 Complete [00h 03m 15s]
val_loss: 0.015220738016068935

Best val_loss So Far: 0.012443347834050655
Total elapsed time: 00h 37m 09s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt_Dropout
Showing 10 best trials
Trial summary
Hyperparameters:
units: 6
Dropout_rate: 0.15000000000000002
Score: 0.012443347834050655
Trial summary
Hyperparameters:
units: 10
Dropout_rate: 0.15000000000000002
Score: 0.014988685958087444
Trial summary
Hyperparameters:
units: 4
Dropout_rate: 0.4
Score: 0.015047373250126839
Trial summary
Hyperparameters:
units: 6
Dropout_rate: 0.4
Score: 0.015220738016068935
Trial summary
Hyperparameters:
units: 8
Dropout_rate: 0.0
Score: 0.015297350473701954
Trial summary
Hyperparameters:
units: 32
Dropout_rate: 0.35000000000000003
Score: 0.01544713880866766
Trial summary
Hyperparameters:
units: 18
Dropout_rate: 0.4
Score: 0.015642007812857628
Trial summary
Hyperparameters:
units: 22
Dropout_rate: 0.30000000000000004
Score: 0.01723344437777996
Trial summary
Hyperparameters:
units: 10
Dropout_rate: 0.1
Score: 0.01819143258035183
Trial summary
Hyperparameters:
units: 32
Dropout_rate: 0.30000000000000004
Score: 0

Learning rate

In [ ]:
tuner.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 3 Complete [00h 03m 17s]
val_loss: 0.05982368811964989

Best val_loss So Far: 0.013453940860927105
Total elapsed time: 00h 10m 10s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/intro_to_kt_LearningRate
Showing 10 best trials
Trial summary
Hyperparameters:
learning_rate: 0.001
Score: 0.013453940860927105
Trial summary
Hyperparameters:
learning_rate: 0.01
Score: 0.020788146182894707
Trial summary
Hyperparameters:
learning_rate: 0.0001
Score: 0.05982368811964989


In [ ]:
tuner.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=10,
    steps_per_epoch = 200,
)

Trial 60 Complete [00h 03m 25s]
val_loss: 0.10991734266281128

Best val_loss So Far: 0.011991373263299465
Total elapsed time: 03h 37m 08s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tunerB.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=20,
    steps_per_epoch = 200,
)

Trial 20 Complete [00h 02m 01s]
val_loss: 0.02421070635318756

Best val_loss So Far: 0.015173292718827724
Total elapsed time: 01h 31m 02s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tunerB.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units LSTM: 20
units Dense: 40
Dropout rate: 0.5
Score: 0.015173292718827724
Trial summary
Hyperparameters:
units LSTM: 80
units Dense: 22
Dropout rate: 0.0
Score: 0.015263075940310955
Trial summary
Hyperparameters:
units LSTM: 100
units Dense: 40
Dropout rate: 0.0
Score: 0.015567285008728504
Trial summary
Hyperparameters:
units LSTM: 20
units Dense: 40
Dropout rate: 0.0
Score: 0.01563265174627304
Trial summary
Hyperparameters:
units LSTM: 20
units Dense: 2
Dropout rate: 0.2
Score: 0.0159071683883667
Trial summary
Hyperparameters:
units LSTM: 20
units Dense: 2
Dropout rate: 0.0
Score: 0.015958935022354126
Trial summary
Hyperparameters:
units LSTM: 120
units Dense: 28
Dropout rate: 0.0
Score: 0.015971561893820763
Trial summary
Hyperparameters:
units LSTM: 20
units Dense: 22
Dropout rate: 0.0
Score: 0.016156133264303207
Trial summary
Hyperparameters:
units LSTM: 120
units Dense: 2
Dropout 

GRU

In [ ]:
tunerB2.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=20,
    steps_per_epoch = 200,
)

Trial 26 Complete [00h 18m 43s]
val_loss: 0.009782456792891026

Best val_loss So Far: 0.009035434573888779
Total elapsed time: 03h 46m 51s

Search: Running Trial #27

Value             |Best Value So Far |Hyperparameter
200               |120               |units LSTM
160               |160               |units Dense
0.2               |0                 |Dropout rate

Epoch 1/20
200/200 [==============================] - 46s 219ms/step - loss: 0.2116 - val_loss: 0.1936
Epoch 2/20
200/200 [==============================] - 42s 208ms/step - loss: 0.0805 - val_loss: 0.0270
Epoch 3/20
200/200 [==============================] - 41s 205ms/step - loss: 0.0545 - val_loss: 0.1142
Epoch 4/20
200/200 [==============================] - ETA: 0s - loss: 0.0426

KeyboardInterrupt: ignored

In [ ]:
tunerB2.results_summary()

Results summary
Results in lstm/untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units LSTM: 120
units Dense: 160
Dropout rate: 0.0
Score: 0.009035434573888779
Trial summary
Hyperparameters:
units LSTM: 120
units Dense: 160
Dropout rate: 0.30000000000000004
Score: 0.00928416382521391
Trial summary
Hyperparameters:
units LSTM: 120
units Dense: 160
Dropout rate: 0.1
Score: 0.009386003948748112
Trial summary
Hyperparameters:
units LSTM: 120
units Dense: 160
Dropout rate: 0.30000000000000004
Score: 0.009432257153093815
Trial summary
Hyperparameters:
units LSTM: 120
units Dense: 160
Dropout rate: 0.1
Score: 0.009477569721639156
Trial summary
Hyperparameters:
units LSTM: 120
units Dense: 160
Dropout rate: 0.5
Score: 0.009523987770080566
Trial summary
Hyperparameters:
units LSTM: 250
units Dense: 160
Dropout rate: 0.1
Score: 0.009744036942720413
Trial summary
Hyperparameters:
units LSTM: 250
units Dense: 80
Dropout rate: 0.2
Score: 0.009782456792891026
Trial summary
Hype

In [ ]:
tunerB3.search(train_gen, 
    validation_data = val_gen,
    validation_steps = val_steps, 
    epochs=20,
    steps_per_epoch = 200,
)

Trial 10 Complete [00h 05m 45s]
val_loss: 0.020187892019748688

Best val_loss So Far: 0.014422482810914516
Total elapsed time: 01h 11m 15s

Search: Running Trial #11

Value             |Best Value So Far |Hyperparameter
125               |125               |units LSTM
150               |150               |units Dense

Epoch 1/20
200/200 [==============================] - 24s 103ms/step - loss: 0.1994 - val_loss: 0.2682
Epoch 2/20
200/200 [==============================] - 20s 98ms/step - loss: 0.0760 - val_loss: 0.0475
Epoch 3/20
200/200 [==============================] - 19s 96ms/step - loss: 0.0619 - val_loss: 0.1218
Epoch 4/20
200/200 [==============================] - 19s 97ms/step - loss: 0.0557 - val_loss: 0.0807
Epoch 5/20
200/200 [==============================] - 19s 97ms/step - loss: 0.0513 - val_loss: 0.0741
Epoch 6/20
200/200 [==============================] - 19s 97ms/step - loss: 0.0492 - val_loss: 0.0635
Epoch 7/20
200/200 [==============================] - 19s 96ms/step

KeyboardInterrupt: ignored